<a href="https://colab.research.google.com/github/regenbuild/BIM2BEM/blob/main/TopologicalGraphDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy
!pip install topologicpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.5/369.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.5 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=aa394426f913f20f26c211b8d57c3066641dac3ea330d8932c448f9a588951fe
  Stored i

In [80]:
from scipy.spatial import ConvexHull, Delaunay, Voronoi, delaunay_plot_2d, convex_hull_plot_2d, voronoi_plot_2d, KDTree

import numpy as np
import pandas as pd

from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex

from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary

import csv
import re

In [4]:
from google.colab import files
uploaded = files.upload()


Saving SlabsWallsZones.csv to SlabsWallsZones.csv


In [ ]:
print(uploaded)

{'zones3DFinal (1).csv': b'IfcSpace/397,-12.7,14.42,0.49,"[3, 0, 1, 2, 3, 3, 4, 5, 3, 3, 5, 1, 3, 6, 1, 0, 3, 6, 3, 1, 3, 7, 8, 4, 3, 7, 4, 3, 3, 9, 7, 3, 3, 10, 11, 8, 3, 10, 8, 7, 3, 12, 11, 10, 3, 13, 14, 12, 3, 13, 12, 10, 3, 15, 14, 13, 3, 16, 15, 13, 3, 17, 18, 15, 3, 17, 15, 16, 3, 19, 20, 6, 3, 19, 6, 0, 3, 0, 2, 21, 3, 0, 21, 19, 3, 22, 21, 2, 3, 22, 2, 1, 3, 1, 5, 23, 3, 1, 23, 22, 3, 23, 5, 4, 3, 24, 23, 4, 3, 24, 4, 8, 3, 24, 8, 25, 3, 26, 25, 8, 3, 26, 8, 11, 3, 27, 26, 11, 3, 27, 11, 12, 3, 28, 27, 12, 3, 28, 12, 14, 3, 29, 28, 14, 3, 29, 14, 15, 3, 30, 29, 15, 3, 30, 15, 18, 3, 18, 17, 31, 3, 18, 31, 30, 3, 32, 31, 17, 3, 32, 17, 16, 3, 33, 32, 16, 3, 33, 16, 13, 3, 10, 33, 13, 3, 10, 34, 33, 3, 7, 34, 10, 3, 7, 35, 34, 3, 35, 7, 9, 3, 35, 9, 36, 3, 37, 36, 9, 3, 37, 9, 3, 3, 3, 6, 20, 3, 3, 20, 37, 3, 22, 19, 21, 3, 20, 19, 22, 3, 37, 20, 22, 3, 23, 37, 22, 3, 24, 37, 23, 3, 35, 36, 37, 3, 35, 37, 24, 3, 25, 35, 24, 3, 34, 35, 25, 3, 26, 34, 25, 3, 27, 33, 34, 3, 27, 34

In [6]:
import io
points_df = pd.read_csv(io.StringIO(uploaded['SlabsWallsZones.csv'].decode('utf-8')), names= ['id', 'name', 'x', 'y', 'z', 'indices'])


In [29]:
print(points_df.head())

                                 id  \
0  536801b326a11a0fabbd0d31cdedeb48   
1  536801b326a11a0fabbd0d31cdedeb48   
2  536801b326a11a0fabbd0d31cdedeb48   
3  536801b326a11a0fabbd0d31cdedeb48   
4  536801b326a11a0fabbd0d31cdedeb48   

                                                name      x      y     z  \
0  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29  10.39  5.41   
1  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29  10.39  0.49   
2  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.44  10.39  0.49   
3  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.44  10.39  5.41   
4  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29   0.09  5.41   

                                             indices  
0  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
1  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
2  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
3  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
4  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4,

# Required Functions

In [9]:
def from_vertices_to_points_3D(vertices):
  return [[vertex.X(), vertex.Y(), vertex.Z()] for vertex in vertices]

def from_points_to_vertices_3D(points):
  return [Vertex.ByCoordinates(point[0], point[1], point[2]) for point in points]

In [10]:
def make_faces_from_indices(points, face_indices):
  l = len(face_indices)
  i = 0
  triangles = []

  while i < l:
    curr = face_indices[i]
    i+=1
    lis = face_indices[i:curr+i]
    triangles.append(lis)

    i+=curr

  faces_list = []

  for triangle in triangles:
    v1 = points[triangle[0]]
    v2 = points[triangle[1]]
    v3 = points[triangle[2]]
    v_list = from_points_to_vertices_3D([v1, v2, v3])
    f = Face.ByVertices(v_list)
    faces_list.append(f)

  return faces_list



In [16]:
WALL_REG = r"(.*IfcWall.*)"
SLAB_REG = r"(.*IfcSlab.*)"
WINDOW_REG = r"(.*IfcWindow.*)"
COLUMN_REG = r"(.*IfcColumn.*)"
DOOR_REG = r"(.*IfcDoor.*)"
SPACE_REG = r"(.*IfcSpace.*)"

# Topology Building

In [111]:
processed = {}

for tup in points_df.itertuples():
  # print(tup[2])
  if re.findall(SLAB_REG, tup[2]) or re.findall(SPACE_REG, tup[2]):
    # processed = {}
    if tup[1] not in processed:
      processed[tup[1]] = [[tup[2], tup[3], tup[4], tup[5], tup[6]]]
    else:
      processed[tup[1]].append([tup[2], tup[3], tup[4], tup[5], tup[6]])

packed = {}

for key, values in processed.items():
  coords = []

  for value in values:
    coords.append([value[1], value[2], value[3]])

  packed[key] = [values[0][0], coords, [int(i) for i in values[0][4][1:-1].split(", ")]]

after_packed = {}

for key, value in packed.items():
  if value[0] not in after_packed:
    after_packed[value[0]] = [[key, value[1], value[2]]]
  else:
    after_packed[value[0]].append([key, value[1], value[2]])

# for key, value in after_packed.items():
#   print(key, value)


ready_els = {}

for key, values in after_packed.items():
  cells = []
  for value in values:
    # print(value)
    c = Cell.ByFaces(make_faces_from_indices(points=value[1], face_indices=value[2]))
    cells.append(c)

  # print(len(cells))

  if len(cells) > 1:
    # print('complex el')
    cc = CellComplex.ByCells(cells)
    simp = CellComplex.ExternalBoundary(cc)
    ready_els[key] = simp
  else:
    # print('simple el')
    ready_els[key] = cells[0]

  # ready_els[key] = cells

In [112]:
Topology.Show(list(ready_els.values()), renderer='colab', vertexSize=4)